In [1]:
##Cannot run as adminstrator, links to network locatiosn won't work
#This should replace almost all of the MS Access work to get David's summary spreadsheet happening


##Robs summariser of RC8 regoinal Contributor tables
import pandas as pd
import numpy as np
import os

#mainPath = 'P:\projects\RC9_ResultsSets\RegContribTables'
mainPath = r'\\athenasmb\pdreef\RC11_RC2020\RC11_RC2020_ResultsSets_PointOfTruth'
outPath = r'P:\projects\RC11_ResultsSets'
summariesOutFolder = 'SummariesWithProcessQldGov'
#summariesOutFolder = 'SummariesWithProcessAusGov'
baseFolderEnd = 'BASE_RC11'
changeFolderEnd = 'CHANGE_RC11'
predevFolderEnd = 'PREDEV_RC11'
regContributorFileName = 'RegContributorDataGrid.csv'
#outputCSVEnd = 'RegToOutlet_Alt.csv'
outputCSVEnd = 'RegToOutlet.csv'
baselineScenarioName = 'Baseline'
changeScenarioName = 'Change'
predevScenarioName = 'PreDev'
reportCardString = 'RC2020'

pathToContstitsToGrpFile = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\constituentsToGroups.csv'

#Must be utilised in regContributors that do NOT include nesting structures
#alternativeRegionsCSV = {'FI':r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\FI_ReportingRegions_Details.csv',
#                        'BU':r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\BU_ReportingRegions_Details.csv'}
alternativeRegionsCSV = {}

altRegLinker = 'Catchmt'
standardRegName = 'RepReg'
altRegColName = 'AUSGMCAS'
origLinker = 'ModelElement'

annLoadToStream = 'AnnLoadToStreamKG'
annLoadToExport = 'AnnLoadToExportKG'

#For QldGov
#regionIDs = ['BU', 'BM', 'CY', 'FI', 'MW', 'WT']
regionIDs = {'BU':'Burdekin', 'BM':'Burnett Mary', 'CY':'Cape York', 'FI':'Fitzroy', 'MW':'Mackay Whitsunday', 'WT':'Wet Tropics'}

#For AusGov
#regionIDs = {'BU':'Burdekin', 'FI':'Fitzroy'}
#regionIDs = ['BU','FI']
#regionIDs = ['MW']

scenarioToFolderDict = {baselineScenarioName:baseFolderEnd, changeScenarioName:changeFolderEnd, predevScenarioName:predevFolderEnd}


In [2]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)

In [3]:
if not os.path.exists(outPath + '\\' + summariesOutFolder):
    os.makedirs(outPath + '\\' + summariesOutFolder)
    print("Made dir: " + outPath + '\\' + summariesOutFolder)

In [4]:
def getPathInfo(regionIDString, scenarioString):
    fileIn = mainPath + '\\' + regionIDString + '\\Model_Outputs\\' + regionIDString + '_' + scenarioToFolderDict[scenarioString] + '\\' + regContributorFileName
    fileOut = outPath + '\\' + summariesOutFolder + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    return fileIn, fileOut


In [5]:
def produceSummaryFromRegContrib(regionIDString, scenarioName):
    
    theFileIn, theFileOut = getPathInfo(regionIDString, scenarioName)
    
    print("Processing " + theFileIn + " Scenario: " + scenarioName)
    
    rawcontrib = pd.read_csv(theFileIn)
    
    rawcontrib['SCENARIO'] = scenarioName
    rawcontrib[annLoadToStream] = rawcontrib['LoadToStream (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
    rawcontrib[annLoadToExport] = rawcontrib['LoadToRegExport (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
    rawcontrib['AreaHA'] = rawcontrib['AreaM2'].div(10000)
    
    if regionIDString in alternativeRegionsCSV:
        print("Reading alternative regionalistaion for " + regionIDString)
        altDetsDF = pd.read_csv(alternativeRegionsCSV[regionIDString])
        combinedDF = pd.merge(rawcontrib, altDetsDF, how='left', left_on=[origLinker], right_on = [altRegLinker])
        #Transfer vals
        combinedDF['Rep_Region'] = combinedDF[altRegColName]
        #Drop cols
        #where 1 is the axis number (0 for rows and 1 for columns.)
        combinedDF = combinedDF.drop(altRegColName, 1)
        combinedDF = combinedDF.drop(standardRegName, 1)
        combinedDF = combinedDF.drop(altRegLinker, 1)
        rawcontrib = combinedDF
    
    ### Can keep reporting region in the area table, as this is our Non-Nested notebook
    catchFuArea = pd.DataFrame(rawcontrib.groupby(['Rep_Region','ModelElement','FU']).agg({'AreaHA':'first'})).reset_index()
    regFuArea = pd.DataFrame(catchFuArea.groupby(['Rep_Region','FU']).agg({'AreaHA':'sum'})).reset_index()
    
    #regLuseSummary = pd.DataFrame(rawcontrib[rawcontrib['Constituent'].isin(selectedConstituents)].groupby(['Rep_Region','Constituent','FU','Process']).agg({'AnnLoadToStreamKG':'sum','AnnLoadToExportKG':'sum'})).reset_index()
    regLuseSummary = pd.DataFrame(rawcontrib.groupby(['SCENARIO','Rep_Region','Constituent','FU','Process']).agg({annLoadToStream:'sum',annLoadToExport:'sum'})).reset_index()
    
    ### Join/merge with Regional Areas
    regLuseSumPlusArea = pd.merge(regLuseSummary, regFuArea, how='left', left_on=['Rep_Region','FU'], right_on = ['Rep_Region','FU'])
    
    regLuseSumPlusArea.to_csv(theFileOut, index=False)
    
    print("Saved " + theFileOut)
    
    return

    

In [6]:
#List out the processes

for theReg in regionIDs.keys():
    
    for scenario in scenarioToFolderDict.keys():
        
        print("Doing: " + theReg + " " + scenario)
        produceSummaryFromRegContrib(theReg, scenario)



print("Finished first summary")

Doing: BU Baseline
Processing \\athenasmb\pdreef\RC11_RC2020\RC11_RC2020_ResultsSets_PointOfTruth\BU\Model_Outputs\BU_BASE_RC11\RegContributorDataGrid.csv Scenario: Baseline
Saved P:\projects\RC11_ResultsSets\SummariesWithProcessQldGov\BU_RC2020_Baseline_RegToOutlet.csv
Doing: BU Change
Processing \\athenasmb\pdreef\RC11_RC2020\RC11_RC2020_ResultsSets_PointOfTruth\BU\Model_Outputs\BU_CHANGE_RC11\RegContributorDataGrid.csv Scenario: Change
Saved P:\projects\RC11_ResultsSets\SummariesWithProcessQldGov\BU_RC2020_Change_RegToOutlet.csv
Doing: BU PreDev
Processing \\athenasmb\pdreef\RC11_RC2020\RC11_RC2020_ResultsSets_PointOfTruth\BU\Model_Outputs\BU_PREDEV_RC11\RegContributorDataGrid.csv Scenario: PreDev
Saved P:\projects\RC11_ResultsSets\SummariesWithProcessQldGov\BU_RC2020_PreDev_RegToOutlet.csv
Doing: BM Baseline
Processing \\athenasmb\pdreef\RC11_RC2020\RC11_RC2020_ResultsSets_PointOfTruth\BM\Model_Outputs\BM_BASE_RC11\RegContributorDataGrid.csv Scenario: Baseline
Saved P:\projects\RC1

In [7]:
#Build relationships of constituent names to groups
#tssGrp = 'TSS'
#tnGrp = 'TN'
#tpGrp = 'TP'
#ps2Grp = 'PSII'
#nonps2Grp = 'non-PSII'
##Nope, will read in a CSV to DataFrame instead, thus joing will 'drop' flow from this comparison
#constToGrp = {'Sediment - Fine':tssGrp}

constToGrp = pd.read_csv(pathToContstitsToGrpFile)

baseSupplyStr = 'Base_Supply_KG'
baseExportStr = 'Base_Export_KG'
changeSupplyStr = 'Change_Supply_KG'
changeExportStr = 'Change_Export_KG'
predevSupplyStr = 'PreDev_Supply_KG'
predevExportStr = 'PreDev_Export_KG'
baseSupplyArealStr = 'Base_Supply_KGperHA'
baseExportArealStr = 'Base_Export_KGperHA'
changeSupplyArealStr = 'Change_Supply_KGperHA'
changeExportArealStr = 'Change_Export_KGperHA'
predevSupplyArealStr = 'PreDev_Supply_KGperHA'
predevExportArealStr = 'PreDev_Export_KGperHA'
repRegStr = 'Rep_Region'
sourceRegStr = 'Source Region'
constituentStr = 'Constituent'
fuStr = 'FU'
AreaHA_Str = 'AREA_HA'
groupStr = 'GROUP'
modelRegStr = 'MODELREG'
percRedStr = 'PercReduction'
regTotalStr = 'REGIONAL TOTAL'
processStr = 'Process'

#allExportsAndReductions = pd.DataFrame(columns=[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr])
#allExportsAndReductions = pd.DataFrame()


In [8]:
#So now we would be reading in known summary files

def calculateReductions(regName, thisRegID):
    
    unneededFile, theBaseSummaryFile = getPathInfo(thisRegID, baselineScenarioName)
    unneededFile, theChangeSummaryFile = getPathInfo(thisRegID, changeScenarioName)
    unneededFile, thePreDevSummaryFile = getPathInfo(thisRegID, predevScenarioName)
    
    #print("Reading: " + theBaseSummaryFile)
    baseSummaryDF = pd.read_csv(theBaseSummaryFile)
    #print("Reading: " + theChangeSummaryFile)
    changeSummaryDF = pd.read_csv(theChangeSummaryFile)
    #print("Reading: " + thePreDevSummaryFile)
    predevSummaryDF = pd.read_csv(thePreDevSummaryFile)
    
    #inner drops out those that don't merge, left keep them
    baseTotals = pd.merge(baseSummaryDF.groupby([repRegStr,constituentStr,fuStr,processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum', 'AreaHA':'first'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT']).reset_index()
    baseTotals = baseTotals.rename(columns={repRegStr:sourceRegStr,'AreaHA':'AREA_HA',annLoadToStream:baseSupplyStr, annLoadToExport:baseExportStr})
    baseTotals = baseTotals[[sourceRegStr, groupStr, constituentStr, fuStr, processStr, 'AREA_HA', baseSupplyStr, baseExportStr]]
    #baseTotals.head(40)
    
    predevTotals = pd.merge(predevSummaryDF.groupby([repRegStr,constituentStr,fuStr,processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT']).reset_index()
    predevTotals = predevTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:predevSupplyStr, annLoadToExport:predevExportStr})
    predevTotals = predevTotals[[sourceRegStr, constituentStr, fuStr, processStr, predevSupplyStr, predevExportStr]]
    
    changeTotals = pd.merge(changeSummaryDF.groupby([repRegStr,constituentStr,fuStr,processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT'])
    changeTotals = changeTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:changeSupplyStr, annLoadToExport:changeExportStr})
    changeTotals = changeTotals[[sourceRegStr, constituentStr, fuStr, processStr, changeSupplyStr, changeExportStr]]
    #changeTotals.head(8)
    
    #, columns=['Rep_Region', 'GROUP', 'Constituent', 'FU', 'AREA_HA', 'Base_Supply_KG', 'Base_Export_KG', 'PreDev_Supply_KG', 'PreDev_Export_KG', 'Change_Supply_KG', 'Change_Export_KG']
    anthroRegByLuseProcess = pd.merge(pd.merge(baseTotals, predevTotals, how='left', left_on=[sourceRegStr,constituentStr,fuStr,processStr],
                                        right_on = [sourceRegStr,constituentStr,fuStr,processStr]).reset_index(), changeTotals, how='left',
                               left_on=[sourceRegStr,constituentStr,fuStr,processStr],
                               right_on = [sourceRegStr,constituentStr,fuStr,processStr])
    #anthroRegByLuse.head(8)
    
    theFileOut = outPath + '\\' + summariesOutFolder + '\\' + thisRegID + '_RC2020_AnthroRegByLanduseProcess.csv'
    anthroRegByLuseProcess.to_csv(theFileOut, index=False)
    #copied = anthroRegByLuseProcess.copy()
    anthroRegByLuse = anthroRegByLuseProcess.groupby([sourceRegStr, constituentStr, groupStr, fuStr]).agg({'AREA_HA':'max',
                                        baseSupplyStr:'sum', baseExportStr:'sum', predevSupplyStr:'sum',
                                        predevExportStr:'sum', changeSupplyStr:'sum', changeExportStr:'sum'}).reset_index()
    
    #Set these all to zero
    anthroRegByLuse[baseSupplyArealStr] = 0
    anthroRegByLuse[baseExportArealStr] = 0
    anthroRegByLuse[predevSupplyArealStr] = 0
    anthroRegByLuse[predevExportArealStr] = 0
    anthroRegByLuse[changeSupplyArealStr] = 0
    anthroRegByLuse[changeExportArealStr] = 0

    #Now calc when area > zero
    
#     anthroRegByLuse.loc[anthroRegByLuse['AREA_HA'] > 0,
#         baseSupplyArealStr, baseExportArealStr,
#         predevSupplyArealStr, predevExportArealStr,
#         changeSupplyArealStr, changeExportArealStr] = anthroRegByLuse[baseSupplyStr] / anthroRegByLuse['AREA_HA'], anthroRegByLuse[baseExportStr] / anthroRegByLuse['AREA_HA'], anthroRegByLuse[predevSupplyStr] / anthroRegByLuse['AREA_HA'], anthroRegByLuse[predevExportStr] / anthroRegByLuse['AREA_HA'], anthroRegByLuse[changeSupplyStr] / anthroRegByLuse['AREA_HA'], anthroRegByLuse[changeExportStr] / anthroRegByLuse['AREA_HA']
    
       
    anthroRegByLuse.loc[anthroRegByLuse['AREA_HA'] > 0, baseSupplyArealStr] = anthroRegByLuse[baseSupplyStr] / anthroRegByLuse['AREA_HA']
    anthroRegByLuse.loc[anthroRegByLuse['AREA_HA'] > 0, baseExportArealStr] = anthroRegByLuse[baseExportStr] / anthroRegByLuse['AREA_HA']
    anthroRegByLuse.loc[anthroRegByLuse['AREA_HA'] > 0, predevSupplyArealStr] = anthroRegByLuse[predevSupplyStr] / anthroRegByLuse['AREA_HA']
    anthroRegByLuse.loc[anthroRegByLuse['AREA_HA'] > 0, predevExportArealStr] = anthroRegByLuse[predevExportStr] / anthroRegByLuse['AREA_HA']
    anthroRegByLuse.loc[anthroRegByLuse['AREA_HA'] > 0, changeSupplyArealStr] = anthroRegByLuse[changeSupplyStr] / anthroRegByLuse['AREA_HA']
    anthroRegByLuse.loc[anthroRegByLuse['AREA_HA'] > 0, changeExportArealStr] = anthroRegByLuse[changeExportStr] / anthroRegByLuse['AREA_HA']
    
    theLuseFileOut = theFileOut.replace("AnthroRegByLanduseProcess", 'AnthroRegByLanduse')
    
    #Add this col
    anthroRegByLuse[modelRegStr] = regName
    
    #reorder cols
    anthroRegByLuse = anthroRegByLuse[[modelRegStr, sourceRegStr, groupStr, constituentStr, fuStr, AreaHA_Str,
                                  baseSupplyStr, baseExportStr, predevSupplyStr, predevExportStr, changeSupplyStr, changeExportStr,
                                 baseSupplyArealStr, baseExportArealStr, predevSupplyArealStr, predevExportArealStr, changeSupplyArealStr, changeExportArealStr]]
    
    anthroRegByLuse.to_csv(theLuseFileOut, index=False)
    
    #print("Saved " + theFileOut)
    
    regExports = anthroRegByLuseProcess.groupby([sourceRegStr, constituentStr, groupStr]).agg({predevExportStr:'sum', baseExportStr:'sum', changeExportStr:'sum'})
    
    regExports[modelRegStr] = regName
    #print(regExports.head(15))
    regExports[percRedStr] = (regExports[baseExportStr] - regExports[changeExportStr]).div(regExports[baseExportStr] - regExports[predevExportStr]).mul(100)
    regExports = regExports.reset_index()
    regExports = regExports[[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr]]
    #regExports.head(15)
    
    modelRegExports = regExports.groupby([modelRegStr, constituentStr, groupStr]).agg({predevExportStr:'sum', baseExportStr:'sum', changeExportStr:'sum'})
    modelRegExports[sourceRegStr] = regTotalStr
    modelRegExports[percRedStr] = (modelRegExports[baseExportStr] - modelRegExports[changeExportStr]).div(modelRegExports[baseExportStr] - modelRegExports[predevExportStr]).mul(100)
    modelRegExports = modelRegExports.reset_index()
    modelRegExports = modelRegExports[[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr]]
    ##modelRegExports.head(15)
    regExports = regExports.append(modelRegExports, ignore_index=True)
    
    return regExports


In [9]:
allExportsAndReductions = pd.DataFrame(columns=[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr])

for theReg in regionIDs.keys():
    print("Doing: " + theReg + " reductions")
    theseExports = calculateReductions(regionIDs[theReg], theReg)
    allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# ##First 2 for AusGov Regions
# theseExports = calculateReductions('Burdekin', 'BU')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Fitzroy', 'FI')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)


# theseExports = calculateReductions('Burnett Mary', 'BM')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Cape York', 'CY')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)


# theseExports = calculateReductions('Mackay Whitsunday', 'MW')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Wet Tropics', 'WT')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theFileNameOut = outPath + '\\' + summariesOutFolder + '\\AllExportsAndReductions_FromProcess.csv'
allExportsAndReductions.to_csv(theFileNameOut, index=False)
allExportsAndReductions.head(20)
#print(allExportsAndReductions)

Doing: BU reductions
Doing: BM reductions
Doing: CY reductions
Doing: FI reductions
Doing: MW reductions
Doing: WT reductions


,MODELREG,Source Region,Constituent,GROUP,PreDev_Export_KG,Base_Export_KG,Change_Export_KG,PercReduction
0,Burdekin,Black,N_DIN,TN,3.643923e+04,5.556907e+04,5.556907e+04,0.000000
1,Burdekin,Black,N_DON,TN,1.712119e+05,1.807114e+05,1.807114e+05,0.000000
2,Burdekin,Black,N_Particulate,TN,1.411692e+05,1.811222e+05,1.810941e+05,0.070349
3,Burdekin,Black,P_DOP,TP,7.452347e+03,7.936113e+03,7.936113e+03,0.000000
4,Burdekin,Black,P_FRP,TP,2.146319e+04,2.344667e+04,2.344667e+04,0.000000
5,Burdekin,Black,P_Particulate,TP,5.000960e+04,6.248308e+04,6.247233e+04,0.086157
6,Burdekin,Black,Sediment - Fine,TSS,2.630476e+07,4.916443e+07,4.914403e+07,0.089232
7,Burdekin,Burdekin,N_DIN,TN,8.228726e+05,1.136815e+06,1.125400e+06,3.635836
8,Burdekin,Burdekin,N_DON,TN,2.627642e+06,2.721569e+06,2.721569e+06,0.000000
9,Burdekin,Burdekin,N_Particulate,TN,1.291927e+06,3.922681e+06,3.908609e+06,0.534894
